In [38]:
from SCPDefinitions import *
from SCPConstructive import *
from SCPLocalSearch import *
from SCP_solve_all import *
from SCP_GRASP import *

# Instance Representation — `SCPInstance`

Given an SCP instance:

- $m$: number of attributes (elements to cover)  
- $n$: number of sets (airplanes)  
- $\mathbf{c} = [c_1, c_2, \dots, c_n]$: cost of each airplane  
- Coverage is stored implicitly using sparse mappings:

  - `attr_of_set[j]` → attributes covered by set \( j \)  
    (corresponds to column \( j \) of \( A \))  

  - `sets_of_attr[i]` → sets that cover attribute \( i \)  
    (corresponds to row \( i \) of \( A \))

Thus,

$$
a_{ij} = 1 \iff i \in \text{attr\_of\_set}[j] \iff j \in \text{sets\_of\_attr}[i].
$$

The instance defines the data for the optimization model but does not explicitly store matrix \(A\).


In [6]:
inst = SCPInstance(3)
inst.summary()

📘 Instance: scp45.txt
  Attributes (m): 200
  Airplanes (n):  1000
  Known optimal cost: 512.0
Costs sample:
 [1, 1, 1, 1, 1, 1] ...
Example coverage:
  Attribute 0: covered by [774, 7, 779, 656, 274, 541, 798, 290]
  Attribute 1: covered by [130, 261, 393, 400, 657, 532, 155, 412]
  Attribute 2: covered by [2, 34, 35, 930, 166, 302, 814, 833]
  Attribute 3: covered by [260, 518, 647, 649, 905, 532, 32, 160]
Example airplane coverage:
  Airplane 0: covers [68, 75, 21, 127, 159]
  Airplane 1: covers [177, 91, 139, 195]
  Airplane 2: covers [104, 2, 154, 63]
  Airplane 3: covers [89, 34, 119]


# Solution representation

A solution is represented by a binary vector of size n, where each position i indicates whether subset i is included in the cover (1) or not (0).

Given an SCP instance defined by:

- $m$: number of attributes (elements to cover)  
- $n$: number of sets (airplanes)  
- $\mathbf{c} = [c_1, c_2, \dots, c_n]$: cost vector  
- $A \in \{0,1\}^{m \times n}$: coverage matrix with entries  
  $$
  a_{ij} =
  \begin{cases}
  1, & \text{if attribute } i \text{ is covered by set } j, \\
  0, & \text{otherwise.}
  \end{cases}
  $$

---

### **Decision Variables**

Each airplane (set) $j$ has a binary decision variable:
$$
x_j =
\begin{cases}
1, & \text{if airplane } j \text{ is selected,} \\
0, & \text{otherwise.}
\end{cases}
$$

In code:
- `self.selected`  →  $\{\,j : x_j = 1\,\}$  
- `self.covered[i]`  →  $\displaystyle \sum_{j=1}^{n} a_{ij} x_j$  
- `self.cost`  →  $\displaystyle \sum_{j=1}^{n} c_j x_j$

---

### **Objective Function**

Minimize the total cost of selected airplanes:
$$
\min_{x \in \{0,1\}^n} Z = \sum_{j=1}^{n} c_j x_j
$$

---

### **Feasibility Constraints**

Each attribute must be covered by at least one selected set:
$$
\sum_{j=1}^{n} a_{ij} x_j \ge 1, \quad \forall\, i = 1,\dots,m.
$$

In code:  
`is_feasible()` → checks `np.all(self.covered > 0)`

---




In [7]:
inst = SCPInstance(3)
sol = SCPSolution(inst)

sol.add(5)
sol.add(10)
sol.add(200)
sol.remove(10)

sol.summary()
sol.prune_by_cost()
sol.summary()


✈️  SCP Solution Summary
  Selected airplanes: 2
  Total cost: 20.00
  Feasible: False
  Uncovered attributes: 196
Selected (sample): [5, 200]
Uncovered (sample): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

✈️  SCP Solution Summary
  Selected airplanes: 2
  Total cost: 20.00
  Feasible: False
  Uncovered attributes: 196
Selected (sample): [5, 200]
Uncovered (sample): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]



In [8]:
#Load from csv
inst = SCPInstance(0, folder="SCP-Instances")
sol = SCPSolution.from_csv(inst, "results/greedy_RE.csv")

sol.summary()


✅ Loaded solution for scp42: 66 sets, feasible=True, cost=529.00
✈️  SCP Solution Summary
  Selected airplanes: 66
  Total cost: 529.00
  Feasible: True
  Uncovered attributes: 0
Selected (sample): [0, 2, 3, 4, 5, 6, 7, 9, 10, 12]



# CONSTRUCTION HEURISTIC

In [9]:
from SCPConstructive import *

### ✈️ CH0: Greedy First Fit Heuristic

<span style="color:red">Not being used</span>

A simple **constructive heuristic** that builds a feasible solution step by step.

**Idea:**  
Start with no airplanes selected. Repeatedly pick the first uncovered attribute \( i \),  
choose the first set \( j \) that covers it, set \( x_j \leftarrow 1 \),  
update coverage, and stop when all attributes are covered.

Formally, at each iteration:

$$
j^{*} = \min \{\, j : a_{ij} = 1 \,\}, \qquad
i = \text{first uncovered attribute}.
$$

Then set \( x_{j^{*}} = 1 \).

**Characteristics:**  
- Deterministic  
- Greedy and myopic (covers one attribute at a time)  
- Produces a feasible solution, not necessarily optimal  
- A pruning step can later remove redundant sets


In [10]:
inst = SCPInstance(0)  # load first file in folder
sol = greedy_first_fit(inst)
sol.summary()
sol.prune_by_cost()
sol.summary()


✈️  SCP Solution Summary
  Selected airplanes: 81
  Total cost: 616.00
  Feasible: True
  Uncovered attributes: 0
Selected (sample): [0, 1, 2, 4, 5, 6, 7, 9, 10, 11]

✈️  SCP Solution Summary
  Selected airplanes: 70
  Total cost: 572.00
  Feasible: True
  Uncovered attributes: 0
Selected (sample): [0, 1, 2, 4, 5, 6, 7, 9, 10, 12]



### 💰 Ch1 : Greedy Cost-Efficient Heuristic

A **cost-aware constructive heuristic** that builds a feasible solution by balancing coverage and cost.

**Idea:**  
At each step, select the airplane \( j \) that minimizes the ratio between its cost and  
the number of *new* uncovered attributes it would cover.

Formally:

$$
h(j) = \frac{c_j}{|\{\, i : a_{ij} = 1 \text{ and } \text{covered}[i] = 0 \,\}|}
$$

At each iteration:

$$
j^{*} = \arg\min_{j \notin \text{selected}} h(j)
\quad \Longrightarrow \quad x_{j^{*}} = 1.
$$

Repeat until all attributes are covered.

**Characteristics:**  
- Deterministic  
- Balances cost vs. coverage gain  
- More informed than pure greedy-first-fit  
- Still locally myopic — does not look ahead


In [11]:
inst = SCPInstance(0)
sol = greedy_cost_efficiency(inst)
sol.summary()
sol.prune_by_cost()
sol.summary()


✈️  SCP Solution Summary
  Selected airplanes: 81
  Total cost: 582.00
  Feasible: True
  Uncovered attributes: 0
Selected (sample): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

✈️  SCP Solution Summary
  Selected airplanes: 66
  Total cost: 529.00
  Feasible: True
  Uncovered attributes: 0
Selected (sample): [0, 2, 3, 4, 5, 6, 7, 9, 10, 12]



### 💰 CH2: Greedy Cost Squared over Cover Heuristic

A **cost-sensitive constructive heuristic** that increases the penalty on expensive sets while still favoring wide coverage.

**Idea:**  
At each step, select the airplane \( j \) that minimizes the ratio between the **square of its cost** and  
the number of *new* uncovered attributes it would cover.

Formally:

$$
h(j) = \frac{c_j^2}{|\{\, i : a_{ij} = 1 \text{ and } \text{covered}[i] = 0 \,\}|}
$$

At each iteration:

$$
j^{*} = \arg\min_{j \notin \text{selected}} h(j)
\quad \Longrightarrow \quad x_{j^{*}} = 1.
$$

Repeat until all attributes are covered.

**Characteristics:**  
- Deterministic  
- Applies a quadratic cost penalty (discourages expensive sets)  
- Favors multiple cheap sets over single costly ones  
- More conservative than the standard cost-efficiency heuristic  
- Still locally greedy — no lookahead or randomness


In [12]:
inst = SCPInstance(0)
sol = greedy_cost_square_over_cover(inst)
sol.summary()
sol.prune_by_cost()
sol.summary()


✈️  SCP Solution Summary
  Selected airplanes: 88
  Total cost: 639.00
  Feasible: True
  Uncovered attributes: 0
Selected (sample): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

✈️  SCP Solution Summary
  Selected airplanes: 69
  Total cost: 558.00
  Feasible: True
  Uncovered attributes: 0
Selected (sample): [0, 2, 3, 4, 5, 6, 7, 9, 10, 12]



### 🎲 CH3: Greedy Randomized Adaptive Heuristic (GRASP Constructive Phase)

A **stochastic extension** of the greedy cost-efficient heuristic.

At each step:
- Compute the efficiency of each set  
  \( h(j) = \dfrac{c_j}{|\{\, i : a_{ij} = 1 \text{ and } \text{covered}[i] = 0 \,\}|} \)
- Sort sets by efficiency (lower is better)
- Build a **Restricted Candidate List (RCL)** containing the best candidates:
  $$
  \text{RCL} = \{\, j : h(j) \le h_{\min} + \alpha (h_{\max} - h_{\min}) \,\}
  $$
- Randomly pick one set \( j^* \in \text{RCL} \)
- Add that set to the solution: \( x_{j^*} \leftarrow 1 \)
- Update covered attributes

Repeat until all attributes are covered.

**Parameter:**
- \( \alpha \in [0,1] \): controls greediness  
  - \( \alpha = 0 \): purely greedy  
  - \( \alpha = 1 \): purely random  

**Characteristics:**  
- Randomized but adaptive to current coverage  
- Balances exploration (randomness) and exploitation (greedy choice)  
- Provides diverse starting solutions for local search


In [19]:
inst = SCPInstance(0)
sol = greedy_randomized_adaptive(inst, alpha=0.1)
sol.summary()
sol.prune_by_cost()
sol.summary()


✈️  SCP Solution Summary
  Selected airplanes: 79
  Total cost: 1028.00
  Feasible: True
  Uncovered attributes: 0
Selected (sample): [1, 3, 6, 7, 9, 10, 11, 16, 17, 19]

✈️  SCP Solution Summary
  Selected airplanes: 62
  Total cost: 827.00
  Feasible: True
  Uncovered attributes: 0
Selected (sample): [1, 3, 7, 10, 11, 16, 17, 19, 20, 21]



# Look ahead heuristics
<span style="color:red">Not being used</span>

Non-myopic heuristics that consider future consequences of current choices.

\textcolor{orange}{To Be improved}

In [34]:
df_greedy = solve_all_instances("greedy_cost_efficiency", "greedy.csv")
df_greedy_RE = solve_all_instances("greedy_RE", "greedy_RE.csv") 

Solver: greedy_cost_efficiency
Instance 12/42

KeyboardInterrupt: 

In [21]:
df_squared = solve_all_instances("greedy_cost_square_over_cover", "greedy_squared_results.csv")
df_squared_RE = solve_all_instances("squared_RE", "squared_RE_results.csv")

Solver: greedy_cost_square_over_cover
Instance 42/42

==== Summary ====
Solver: greedy_cost_square_over_cover
Average deviation from optimum : +22.94%
Minimum deviation              : +13.79%
Maximum deviation              : +34.43%
Optimal solutions found        : 0/42
Time [fastest, slowest]        : 0.16s  , 1.71s
Average time per instance      : 0.71s
Total elapsed time             : 30.20s
Solver: squared_RE
Instance 42/42

==== Summary ====
Solver: squared_RE
Average deviation from optimum : +7.51%
Minimum deviation              : +0.00%
Maximum deviation              : +18.01%
Optimal solutions found        : 1/42
Time [fastest, slowest]        : 0.17s  , 1.79s
Average time per instance      : 0.73s
Total elapsed time             : 31.30s


In [29]:
df_random = solve_all_instances("greedy_randomized_adaptive", "random.csv", alpha=0.0)
df_random_RE = solve_all_instances("randomized_RE", "random_RE.csv", alpha=0.0)

Solver: greedy_randomized_adaptive
Instance 42/42

==== Summary ====
Solver: greedy_randomized_adaptive
Average deviation from optimum : +13.62%
Minimum deviation              : +8.33%
Maximum deviation              : +23.33%
Optimal solutions found        : 0/42
Time [fastest, slowest]        : 0.16s  , 2.11s
Average time per instance      : 0.73s
Total elapsed time             : 31.22s
Solver: randomized_RE
Instance 24/42

KeyboardInterrupt: 

# Local Search 

Tasks:

Consider one neighbourhood

Implement first-improvement (FI) and best-improvement (BI) algorithms for the SCP. 
 
In these two algorithms, consider one neighborhoods of your choice. Apply redundancy elimination after each step.

Apply each of these algorithms once to an initial solution generated by CH1, CH2, CH3, and CH1+RE.  H

ence, in total eight algorithms should be tested obtained by the combinations of the four constructive heuristics with the two iterative improvement algorithms. 

As variance reduction technique for the experiments make sure that the FI and BI algorithms start from the same initial solution. This can be ensured by using for each of the executions on a same instance the same random number seed. 

As the experimental results report for each of the experiments
the average percentage deviation from best known solutions;
the total computation time across all instances;
the fraction of instances that profit from the additional local search phase.

Determine by means of statistical tests (in this case, the Student t-test or the Wilcoxon test), whether there is a statistically significant difference between the solutions generated by the various algorithms. Consider the data for the Set Covering Problem (SCP) available in Moodle.


# Best Improvement Local Search 1x1

# First Improvement Local Search 1x1

In [ ]:
#example
inst = SCPInstance(0)
#sol.summary()

print("Squared + RE")
sol_squared_RE = greedy_cost_square_over_cover(inst)
sol_squared_RE = sol.copy()
sol_squared_RE.prune_by_cost()
sol_squared_RE.summary()

print("Squared + FI + RE")
sol_squared_FI1x1 = greedy_cost_square_over_cover(inst)
sol_squared_FI1x1 = best_improvement_1x1_loop(sol, max_time=99999)
sol_squared_FI1x1.prune_by_cost()
sol_squared_FI1x1.summary()

print("Squared + BI + RE")
sol_squared_BI1x1 = greedy_cost_square_over_cover(inst)
sol_squared_BI1x1 = best_improvement_1x1_loop(sol, max_time=99999)
sol_squared_BI1x1.prune_by_cost()
sol_squared_BI1x1.summary()



Squared + RE
✈️  SCP Solution Summary
  Selected airplanes: 62
  Total cost: 827.00
  Feasible: True
  Uncovered attributes: 0
Selected (sample): [1, 3, 7, 10, 11, 16, 17, 19, 20, 21]

Squared + FI + RE
✈️  SCP Solution Summary
  Selected airplanes: 59
  Total cost: 675.00
  Feasible: True
  Uncovered attributes: 0
Selected (sample): [0, 1, 3, 4, 5, 6, 7, 14, 17, 18]

Squared + BI + RE
✈️  SCP Solution Summary
  Selected airplanes: 59
  Total cost: 675.00
  Feasible: True
  Uncovered attributes: 0
Selected (sample): [0, 1, 3, 4, 5, 6, 7, 14, 17, 18]



# Running all instances for 1x1 Search

In [35]:
df_greedy_RE = solve_all_instances("greedy_RE", "greedy_RE.csv")
df_greedy_FI_RE = solve_all_instances("greedy_plus_FI1x1", "greedy_FI1x1.csv", fi_time=9999.0)
df_greedy_BI_RE = solve_all_instances("greedy_plus_BI1x1", "greedy_BI1x1.csv", ls_time=9999.0)



Solver: greedy_RE
Instance 42/42

==== Summary ====
Solver: greedy_RE
Average deviation from optimum : +5.52%
Minimum deviation              : +0.47%
Maximum deviation              : +15.53%
Optimal solutions found        : 0/42
Time [fastest, slowest]        : 0.16s  , 1.64s
Average time per instance      : 0.65s
Total elapsed time             : 27.86s
Solver: greedy_plus_FI1x1
Instance 36/42

KeyboardInterrupt: 

In [ ]:
df_squared_RE = solve_all_instances("squared_plus_RE", "squared_RE.csv")
df_greedy_FI_RE = solve_all_instances("squared_plus_FI1x1", "squared_FI1x1.csv", fi_time=9999.0)
df_greedy_BI_RE = solve_all_instances("squared_plus_BI1x1", "squared_BI1x1.csv", ls_time=9999.0)



# Best Improvement Local Search Drop or Swap

# First Improvement Local Search Drop or Swap

In [36]:
#example
inst = SCPInstance(0)
#sol.summary()

print("Squared + RE")
sol = greedy_cost_square_over_cover(inst)
sol_squared_RE = sol.copy()
sol_squared_RE.prune_by_cost()
sol_squared_RE.summary()

print("Squared + FI + RE")
sol_squared_FI1x1 = greedy_cost_square_over_cover(inst)
sol_squared_FI1x1 = first_improvement_drop_or_swap_loop(sol, max_time=99999)
sol_squared_FI1x1.prune_by_cost()
sol_squared_FI1x1.summary()

print("Squared + BI + RE")
sol_squared_BI1x1 = greedy_cost_square_over_cover(inst)
sol_squared_BI1x1 = best_improvement_drop_or_swap_loop(sol, max_time=99999)
sol_squared_BI1x1.prune_by_cost()
sol_squared_BI1x1.summary()



Squared + RE
✈️  SCP Solution Summary
  Selected airplanes: 69
  Total cost: 558.00
  Feasible: True
  Uncovered attributes: 0
Selected (sample): [0, 2, 3, 4, 5, 6, 7, 9, 10, 12]

Squared + FI + RE
✈️  SCP Solution Summary
  Selected airplanes: 70
  Total cost: 559.00
  Feasible: True
  Uncovered attributes: 0
Selected (sample): [0, 2, 3, 4, 5, 6, 7, 9, 10, 12]

Squared + BI + RE
✈️  SCP Solution Summary
  Selected airplanes: 69
  Total cost: 558.00
  Feasible: True
  Uncovered attributes: 0
Selected (sample): [0, 2, 3, 4, 5, 6, 7, 9, 10, 12]



# Running All Instances for Drop or Swap

In [37]:
df_greedy_RE = solve_all_instances("greedy_plus_RE", "greedy_RE.csv")
df_greedy_FI_RE = solve_all_instances("greedy_plus_FI_drop_or_swap", "greedy_FI_drop_or_swap.csv", fi_time=9999.0)
df_greedy_BI_RE = solve_all_instances("greedy_plus_BI_drop_or_swap", "greedy_BI_drop_or_swap.csv", ls_time=9999.0)



Solver: greedy_plus_RE
Instance 42/42

==== Summary ====
Solver: greedy_plus_RE
Average deviation from optimum : +5.52%
Minimum deviation              : +0.47%
Maximum deviation              : +15.53%
Optimal solutions found        : 0/42
Time [fastest, slowest]        : 0.15s  , 1.50s
Average time per instance      : 0.64s
Total elapsed time             : 27.42s
Solver: greedy_plus_FI_drop_or_swap
Instance 16/42

KeyboardInterrupt: 

In [44]:
df_squared_RE = solve_all_instances("squared_plus_RE", "squared_RE.csv")
df_squared_FI_RE = solve_all_instances("squared_plus_FI_drop_or_swap", "squared_FI_drop_or_swap.csv", fi_time=9999.0)
df_squared_BI_RE = solve_all_instances("squared_plus_BI_drop_or_swap", "squared_BI_drop_or_swap.csv", ls_time=9999.0)

Solver: squared_plus_RE
Instance 42/42
Solver: squared_plus_RE Average deviation: +7.51%
Solver: squared_plus_FI_drop_or_swap
Instance 42/42
Solver: squared_plus_FI_drop_or_swap Average deviation: +6.89%


In [43]:
#randomized
df_randomized_RE = solve_all_instances("randomized_plus_RE", "randomized_RE.csv")
df_randomized_FI_RE = solve_all_instances("randomized_plus_FI_drop_or_swap", "randomized_FI_drop_or_swap.csv", alpha = 0.0, fi_time=9999.0)
df_randomized_BI_RE = solve_all_instances("randomized_plus_BI_drop_or_swap", "randomized_BI_drop_or_swap.csv", alpha = 0.0, ls_time=9999.0)

Solver: randomized_plus_BI_drop_or_swap
Instance 42/42
Solver: randomized_plus_BI_drop_or_swap Average deviation: +4.92%


In [47]:
df_greedy_RE_FI = solve_all_instances("greedy_RE_plus_FI_drop_or_swap", "greedy_RE_FI_drop_or_swap.csv", fi_time=9999.0)
df_greedy_RE_BI = solve_all_instances("greedy_RE_plus_BI_drop_or_swap", "greedy_RE_BI_drop_or_swap.csv", ls_time=9999.0)

Solver: greedy_RE_plus_FI_drop_or_swap
Instance 42/42
Solver: greedy_RE_plus_FI_drop_or_swap Average deviation: +4.95%
Solver: greedy_RE_plus_BI_drop_or_swap
Instance 42/42
Solver: greedy_RE_plus_BI_drop_or_swap Average deviation: +4.95%


# Pruning First

In [ ]:
def greedy_RE_BI1X1(instance, ls_time=999.0):
    sol = greedy_cost_efficiency(instance)
    sol.prune_by_cost()
    sol = best_improvement_1x1_loop(sol, max_time=ls_time)
    return sol

def greedy_plus_FI1x1(instance, fi_time=999.0):
    sol = greedy_cost_efficiency(instance)
    sol.prune_by_cost()
    sol = first_improvement_1x1_loop(sol, max_time=fi_time)
    return sol

df_greedy_RE_BI1X1 = solve_all_instances("greedy_RE_BI1X1", "greedy_RE_BI1X1_results.csv", ls_time=9999.0)
df_greedy_RE_FI1X1 = solve_all_instances("greedy_RE_FI1X1", "greedy_RE_FI1X1_results.csv", fi_time=9999.0)

# GRASP : Greedy Randomized Adaptive Search Procedure

## Sequential Execution of GRASP

In [ ]:
small_instance = SCPInstance(0)
large_instance = SCPInstance(33)  # adjust index as needed

small = grasp_sequential(small_instance, alpha=0.05, max_time=300.0, max_iter=10, verbose=True)
large = grasp_sequential(large_instance, alpha=0.05, max_time=300.0, max_iter=10, verbose=True)

small.summary()
large.summary()



🌀 Starting GRASP | alpha=0.05, max_iter=10, max_time=300.0s
   Known optimum for scp42: 512.00

  Iter   1: ✨ New best cost = 558.00 (deviation = +8.98%)
  Iter   3: ✨ New best cost = 541.00 (deviation = +5.66%)
  Iter  10: ✨ New best cost = 525.00 (deviation = +2.54%)
✅ GRASP finished after 10 iterations (33.89s)
   ➤ Best cost found: 525.00 (deviation = +2.54%)


🌀 Starting GRASP | alpha=0.05, max_iter=10, max_time=300.0s
   Known optimum for scpc2: 219.00

  Iter   1: ✨ New best cost = 418.00 (deviation = +90.87%)
  Iter   2: ✨ New best cost = 376.00 (deviation = +71.69%)
  Iter   3: ✨ New best cost = 352.00 (deviation = +60.73%)


## GRASP Parallel Execution

In [ ]:
small_instance = SCPInstance(0)
large_instance = SCPInstance(33)  # adjust index as needed

small = grasp_parallel(small_instance, alpha=0.05, max_time=300.0, max_iter=10, verbose=True)
large = grasp_parallel(large_instance, alpha=0.05, max_time=300.0, max_iter=10, verbose=True)

small.summary()
large.summary()


Solver: grasp_parallel
✅ GRASP Parallel [scp42] best cost=524.00 (dev=+2.34%) in 38.56s using 18 workers.
Instance 1/1
Solver: grasp_parallel Average deviation: +2.34%


# GRASP : Adapting Parameters
## Constant RCL Size

In [21]:
def grasp_single_run_fixed_RCL(instance, desired_RCL=10, max_time=30.0, seed=None):
    """
    Performs one GRASP iteration (randomized construction + local search).
    Designed for parallel execution.
    """
    # Keep RCL size roughly constant across instance sizes
    alpha = min(1.0, desired_RCL / instance.n)

    if seed is not None:
        random.seed(seed)

    # 1️⃣ Construct a randomized solution
    sol = greedy_randomized_adaptive(instance, alpha=alpha, seed=random.randint(0, 1_000_000))
    # 2️⃣ Local search
    sol = first_improvement_drop_or_swap_loop(sol, max_time=max_time)
    # 3️⃣ Redundancy elimination
    sol.prune_by_cost()

    return sol


def grasp_parallel_fixed_RCL(instance, desired_RCL=10, max_time=900.0, num_iter=20, num_workers=4, seed=42):
    """
    Parallel GRASP metaheuristic for the Set Covering Problem.
    Uses a dynamic alpha = desired_RCL / n.
    """
    random.seed(seed)
    best_sol = None
    best_cost = float("inf")
    start = time.time()

    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = [
            executor.submit(
                grasp_single_run_fixed_RCL,
                instance,
                desired_RCL,
                max_time / num_iter,  # optional: split time per run
                random.randint(0, 1_000_000)
            )
            for _ in range(num_iter)
        ]

        for i, f in enumerate(as_completed(futures)):
            sol = f.result()
            if sol.cost < best_cost:
                best_cost = sol.cost
                best_sol = sol

    elapsed = time.time() - start
    opt = getattr(instance, "opt_value", None)
    if opt:
        dev = 100 * (best_cost - opt) / opt
        print(f"✅ GRASP Parallel [{instance.name}] best cost={best_cost:.2f} "
              f"(dev={dev:+.2f}%) in {elapsed:.2f}s using {num_workers} workers.")
    else:
        print(f"✅ GRASP Parallel [{instance.name}] best cost={best_cost:.2f} "
              f"in {elapsed:.2f}s using {num_workers} workers.")

    return best_sol



In [ ]:

#small_instance.summary()
#large_instance.summary()    

grasp_parallel_fixed_RCL(large_instance, desired_RCL=20, max_time=600.0, num_iter=10, num_workers=6, seed=42)
grasp_parallel_fixed_RCL(small_instance, desired_RCL=20, max_time=600.0, num_iter=10, num_workers=6, seed=42)

✅ GRASP Parallel [scpc2] best cost=223.00 (dev=+1.83%) in 39.40s using 6 workers.
✅ GRASP Parallel [scp42] best cost=525.00 (dev=+2.54%) in 7.08s using 6 workers.


In [41]:
solve_all_instances(
    "grasp_parallel_fixed_RCL", 
    "grasp_parallel_fixed_RCL.csv", 
    num_instances=0, 
    desired_RCL=20, 
    max_time=600.0, 
    num_iter=100, 
    num_workers=6, 
    seed=42)


solve_all_instances(
    "grasp_parallel",
    "grasp_parallel_results.csv",
    num_instances=0,
    alpha=0.05,
    max_time=600.0,
    num_iter=100,
    num_workers=6,
    seed=42)




Solver: grasp_parallel_fixed_RCL
✅ GRASP Parallel [scp42] best cost=522.00 (dev=+1.95%) in 45.45s using 6 workers.
Instance 1/42✅ GRASP Parallel [scp43] best cost=520.00 (dev=+0.78%) in 44.39s using 6 workers.
Instance 2/42✅ GRASP Parallel [scp44] best cost=501.00 (dev=+1.42%) in 42.72s using 6 workers.
Instance 3/42✅ GRASP Parallel [scp45] best cost=514.00 (dev=+0.39%) in 44.95s using 6 workers.
Instance 4/42✅ GRASP Parallel [scp46] best cost=563.00 (dev=+0.54%) in 37.95s using 6 workers.
Instance 5/42✅ GRASP Parallel [scp47] best cost=433.00 (dev=+0.70%) in 41.75s using 6 workers.
Instance 6/42✅ GRASP Parallel [scp48] best cost=493.00 (dev=+0.20%) in 38.85s using 6 workers.
Instance 7/42✅ GRASP Parallel [scp49] best cost=652.00 (dev=+1.72%) in 46.36s using 6 workers.
Instance 8/42✅ GRASP Parallel [scp51] best cost=266.00 (dev=+5.14%) in 77.97s using 6 workers.
Instance 9/42✅ GRASP Parallel [scp52] best cost=318.00 (dev=+5.30%) in 75.40s using 6 workers.
Instance 10/42✅ GRASP Parallel

,instance_name,opt_value,solver,solution_cost,deviation_%,time_sec,solution_sets
0,scp42,512.0,grasp_parallel,524.0,2.34,51.7714,"0,1,2,4,5,6,7,9,10,14,16,17,18,19,21,22,24,25,..."
1,scp43,516.0,grasp_parallel,527.0,2.13,53.4007,"0,2,3,7,8,11,12,17,18,19,21,22,23,24,25,26,27,..."
2,scp44,494.0,grasp_parallel,510.0,3.24,52.3412,"0,1,2,3,4,5,7,8,9,10,11,13,15,18,19,20,21,22,2..."
3,scp45,512.0,grasp_parallel,519.0,1.37,51.1856,"0,1,2,5,7,8,11,12,13,17,18,19,20,23,25,27,32,3..."
4,scp46,560.0,grasp_parallel,564.0,0.71,51.2692,"0,1,2,3,4,5,7,10,11,12,13,14,15,17,18,19,20,23..."
5,scp47,430.0,grasp_parallel,433.0,0.70,50.1510,"1,2,3,4,5,12,15,16,17,18,19,20,21,22,25,26,27,..."
6,scp48,492.0,grasp_parallel,499.0,1.42,50.2177,"0,1,2,4,5,6,8,9,10,12,13,14,15,16,17,20,21,22,..."
7,scp49,641.0,grasp_parallel,660.0,2.96,54.2410,"0,1,3,5,8,9,11,13,15,16,17,18,19,20,21,22,25,2..."
8,scp51,253.0,grasp_parallel,271.0,7.11,111.1194,"0,1,2,3,4,5,6,7,10,11,12,13,17,18,19,20,22,23,..."
9,scp52,302.0,grasp_parallel,327.0,8.28,111.6645,"0,1,2,3,4,5,6,7,8,9,10,11,12,15,16,17,18,19,20..."
